# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [27]:
query = """
CREATE TABLE IF NOT EXISTS music_history (
    user_id TEXT,
    session_id TEXT,
    item_in_session TEXT,
    artist TEXT,
    song TEXT,
    first_name TEXT,
    last_name TEXT,
    gender TEXT,
    length TEXT,
    level TEXT,
    location TEXT,
    PRIMARY KEY (user_id, session_id)
);
"""

# Execute the query to create the table
try:
    session.execute(query)
    print("Table 'music_history' has been created successfully.")
except Exception as e:
    print(f"An error occurred while trying to create the table: {e}")

Table 'music_history' has been created successfully.


In [28]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history (artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song, user_id)"			
        query = query + " VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10]))

In [11]:
query = """
CREATE MATERIALIZED VIEW music_history_by_session_view AS
SELECT *
FROM music_history
WHERE session_id IS NOT NULL AND item_in_session IS NOT NULL
PRIMARY KEY ((session_id, item_in_session), user_id);
"""
try:
    session.execute(query)
    print("View 'music_history_by_session_view' has been created successfully.")
except Exception as e:
    print(f"An error occurred while trying to create the table: {e}")

View 'music_history_by_session_view' has been created successfully.


In [13]:
query = "SELECT artist, song, length FROM music_history_by_session_view WHERE session_id='338' AND item_in_session='4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
query = "SELECT * FROM music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.first_name, row.gender, row.item_in_session, row.last_name, row.length, row.level, row.location, row.session_id, row.song, row.user_id)

Jordan Rudess Cecilia F 1 Owens 1367.84934 free Atlanta-Sandy Springs-Roswell, GA 225 Tarkus 6
Ron Van Den Beuken Cecilia F 0 Owens 506.5922 free Atlanta-Sandy Springs-Roswell, GA 229 Timeless 6
Motionless in White Cecilia F 0 Owens 217.57342 free Atlanta-Sandy Springs-Roswell, GA 288 Ghost In The Mirror 6
Phil Collins Cecilia F 0 Owens 239.85587 free Atlanta-Sandy Springs-Roswell, GA 406 The Way You Look Tonight (Live Version) 6
Tub Ring Cecilia F 0 Owens 233.69098 free Atlanta-Sandy Springs-Roswell, GA 423 Invalid 6
Asia 2001 Cecilia F 1 Owens 150.30812 free Atlanta-Sandy Springs-Roswell, GA 444 Epilogue 6
The New Pornographers Cecilia F 1 Owens 173.94893 free Atlanta-Sandy Springs-Roswell, GA 453 Falling Through Your Clothes 6
A Hope For Home Cecilia F 1 Owens 388.38812 free Atlanta-Sandy Springs-Roswell, GA 5 Absolution: Of Flight and Failure 6
Lykke Li Cecilia F 1 Owens 232.35873 free Atlanta-Sandy Springs-Roswell, GA 535 Melodies & Desires 6
Justin Bieber Cecilia F 1 Owens 222.92

As I Lay Dying Chloe F 45 Cuevas 348.1073 paid San Francisco-Oakland-Hayward, CA 1079 Illusions 49
Alliance Ethnik Chloe F 46 Cuevas 195.94404 paid San Francisco-Oakland-Hayward, CA 1096 SinceritÃÂ© Et Jalousie 49
Norther Chloe F 22 Cuevas 246.54322 paid San Francisco-Oakland-Hayward, CA 1114 Frozen Angel 49
Air Chloe F 1 Cuevas 212.21832 free San Francisco-Oakland-Hayward, CA 179 Playground Love 49
Muse Chloe F 3 Cuevas 210.46812 free San Francisco-Oakland-Hayward, CA 195 Pink Ego Box 49
DeGarmo & Key Chloe F 1 Cuevas 236.48608 free San Francisco-Oakland-Hayward, CA 224 I'm Accepted (The Pledge Album Version) 49
mewithoutYou Chloe F 4 Cuevas 311.562 free San Francisco-Oakland-Hayward, CA 267 Bullet To Binary (Pt. Two) 49
Cabaret Voltaire Chloe F 1 Cuevas 430.602 free San Francisco-Oakland-Hayward, CA 274 Playing For Time 49
Carlos Y JosÃÂ© Chloe F 0 Cuevas 166.45179 free San Francisco-Oakland-Hayward, CA 336 Soy AlbaÃÂ±il 49
Ladytron Chloe F 4 Cuevas 325.14567 free San Francisco-Oa

Aesop Rock Anabelle F 1 Simpson 245.65506 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 235 None Shall Pass (Main) 69
The Killers Anabelle F 2 Simpson 245.99465 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 249 Sam's Town 69
A Fine Frenzy Anabelle F 0 Simpson 267.91138 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 256 Almost Lover (Album Version) 69
Alicia Keys Anabelle F 1 Simpson 229.0673 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 259 If I Ain't Got You 69
California Swag District Anabelle F 3 Simpson 239.17669 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 264 Teach Me How To Dougie 69
Laura Izibor Anabelle F 0 Simpson 211.56526 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 378 Carousel (PSILY Album Version) 69
Biffy Clyro Anabelle F 2 Simpson 189.83138 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 455 God & Satan 69
Daft Punk Anabelle F 0 Simpson 396.82567 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 465 Da Funk / Dadftendirekt 69
Iron Maiden Anabelle F 0

Nirvana Aleena F 18 Kirby 257.01832 paid Waterloo-Cedar Falls, IA 637 Lithium 44
Randy Crawford Aleena F 55 Kirby 270.75873 paid Waterloo-Cedar Falls, IA 639 Almaz 44
T.I. Aleena F 23 Kirby 185.10322 paid Waterloo-Cedar Falls, IA 781 Look What I Got (Edited Album Version) 44
Postal Service Jacob M 17 Klein 307.53914 paid Tampa-St. Petersburg-Clearwater, FL 1049 Natural Anthem (Album) 73
The Yardbirds Jacob M 39 Klein 206.10567 paid Tampa-St. Petersburg-Clearwater, FL 255 The Train Kept A Rollin' 73
Los AutÃÂ©nticos Decadentes Jacob M 59 Klein 218.48771 paid Tampa-St. Petersburg-Clearwater, FL 294 La Guitarra 73
Drake / Kanye West / Lil Wayne / Eminem Jacob M 50 Klein 357.66812 paid Tampa-St. Petersburg-Clearwater, FL 518 Forever 73
Dave Brockie Experience Jacob M 49 Klein 116.08771 paid Tampa-St. Petersburg-Clearwater, FL 692 Servant Of Death's Head 73
Beatsteaks Jacob M 27 Klein 177.8673 paid Tampa-St. Petersburg-Clearwater, FL 72 Jane Became Insane 73
Vangelis Jacob M 0 Klein 327.52

Dwight Yoakam Mohammad M 8 Rodriguez 227.05587 paid Sacramento--Roseville--Arden-Arcade, CA 999 Listen (LP Version) 88
B.o.B Tegan F 15 Levine 269.63546 paid Portland-South Portland, ME 1004 Nothin' On You [feat. Bruno Mars] (Album Version) 80
311 Tegan F 40 Levine 113.26649 paid Portland-South Portland, ME 1065 Loco 80
Washed Out Tegan F 3 Levine 168.6722 free Portland-South Portland, ME 165 New Theory 80
The Avett Brothers Tegan F 3 Levine 307.9571 free Portland-South Portland, ME 194 All My Mistakes (US4BW0700133) 80
Ottmar Liebert Tegan F 4 Levine 174.54975 free Portland-South Portland, ME 243 Festival (Of 7 Lights) 80
School Of Seven Bells Tegan F 0 Levine 262.37342 free Portland-South Portland, ME 254 Chain 80
Paul Stanley Tegan F 1 Levine 187.32363 free Portland-South Portland, ME 279 Live To Win 80
Roy Ayers Tegan F 1 Levine 240.90077 free Portland-South Portland, ME 307 Searching 80
Arctic Monkeys Tegan F 1 Levine 152.60689 free Portland-South Portland, ME 329 Baby I'm Yours 8

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
query = """
CREATE MATERIALIZED VIEW music_history_by_item_view AS
SELECT *
FROM music_history
WHERE user_id IS NOT NULL AND session_id IS NOT NULL AND item_in_session IS NOT NULL
PRIMARY KEY ((user_id, session_id), item_in_session);
"""
try:
    session.execute(query)
    print("View 'music_history_by_item_view' has been created successfully.")
except Exception as e:
    print(f"An error occurred while trying to create the table: {e}")

View 'music_history_by_item_view' has been created successfully.


In [16]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT * FROM music_history_by_item_view WHERE user_id='10' AND session_id='182' ORDER BY item_in_session desc"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    full_name = f"{row.first_name} {row.last_name}" 
    print(row.artist, row.song, full_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [35]:
query = """
CREATE MATERIALIZED VIEW music_history_by_song_view AS
SELECT song, user_id, session_id, first_name, last_name
FROM music_history
WHERE song IS NOT NULL AND user_id IS NOT NULL AND session_id IS NOT NULL
PRIMARY KEY (song, user_id, session_id);
"""
try:
    session.execute(query)
    print("View 'music_history_by_song_view' has been created successfully.")
except Exception as e:
    print(f"An error occurred while trying to create the table: {e}")

View 'music_history_by_song_view' has been created successfully.


In [43]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT first_name, last_name FROM music_history_by_song_view WHERE song='All Hands Against His Own' "

try:
    rows = session.execute(query)
    users = set()  
    for row in rows:
        users.add((row.first_name, row.last_name))
    for user in users:
        print(f"{user[0]} {user[1]}")
except Exception as e:
    print(f"An error occurred: {e}")

### Drop the tables before closing out the sessions

In [1]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS music_history"


try:
    session.execute(query)
    print("Table 'music_history' has been dropped successfully.")
except Exception as e:
    print(f"An error occurred while trying to drop the table: {e}")

An error occurred while trying to drop the table: name 'session' is not defined


In [26]:
## TO-DO: Drop the view before closing out the sessions
query = "DROP MATERIALIZED VIEW music_history_by_session_view"


try:
    session.execute(query)
    print("Table 'music_history_by_session_view' has been dropped successfully.")
except Exception as e:
    print(f"An error occurred while trying to drop the table: {e}")

Table 'music_history_by_session_view' has been dropped successfully.


In [44]:
## TO-DO: Drop the view before closing out the sessions
query = "DROP MATERIALIZED VIEW music_history_by_item_view"


try:
    session.execute(query)
    print("Table 'music_history_by_item_view' has been dropped successfully.")
except Exception as e:
    print(f"An error occurred while trying to drop the table: {e}")

Table 'music_history_by_item_view' has been dropped successfully.


In [33]:
## TO-DO: Drop the view before closing out the sessions
query = "DROP MATERIALIZED VIEW music_history_by_song_view"


try:
    session.execute(query)
    print("Table 'music_history_by_song_view' has been dropped successfully.")
except Exception as e:
    print(f"An error occurred while trying to drop the table: {e}")

Table 'music_history_by_song_view' has been dropped successfully.


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()